In [8]:
import torch
from torch import nn
from d2l import torch as d2l
from torch import optim
from tqdm import tqdm

In [2]:
LeNet = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2),
    nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5),
    nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16*5*5, 120),
    nn.Sigmoid(),
    nn.Linear(120, 84),
    nn.Sigmoid(),
    nn.Linear(84, 10)
)
LeNet

Sequential(
  (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): Sigmoid()
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (4): Sigmoid()
  (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=400, out_features=120, bias=True)
  (8): Sigmoid()
  (9): Linear(in_features=120, out_features=84, bias=True)
  (10): Sigmoid()
  (11): Linear(in_features=84, out_features=10, bias=True)
)

In [3]:
X = torch.randn((1, 1, 28, 28), dtype=torch.float32)
for layer in LeNet:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape: \t', X.shape)

Conv2d output shape: 	 torch.Size([1, 6, 28, 28])
Sigmoid output shape: 	 torch.Size([1, 6, 28, 28])
AvgPool2d output shape: 	 torch.Size([1, 6, 14, 14])
Conv2d output shape: 	 torch.Size([1, 16, 10, 10])
Sigmoid output shape: 	 torch.Size([1, 16, 10, 10])
AvgPool2d output shape: 	 torch.Size([1, 16, 5, 5])
Flatten output shape: 	 torch.Size([1, 400])
Linear output shape: 	 torch.Size([1, 120])
Sigmoid output shape: 	 torch.Size([1, 120])
Linear output shape: 	 torch.Size([1, 84])
Sigmoid output shape: 	 torch.Size([1, 84])
Linear output shape: 	 torch.Size([1, 10])


In [4]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

In [9]:
def evaluate_accuracy_gpu(net, data_iter, device='cpu'):
    metric = d2l.Accumulator(2)
    with torch.no_grad():
        for X, y in tqdm(data_iter):
            X = X.to(device)
            y = y.to(device)
            metric.add(d2l.accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [10]:
#@save
def train_ch6(net, train_iter, test_iter, num_epochs, lr, device):
    """用GPU训练模型(在第六章定义)"""
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        # 训练损失之和，训练准确率之和，范例数
        train_loss, train_sum, total_sum = 0.,0.,0.
        net.train()
        for X, y in tqdm(train_iter):
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                train_loss += l
                train_sum += d2l.accuracy(y_hat, y)
                total_sum += len(X)
        train_acc = train_sum / total_sum
        train_loss = train_loss / total_sum
        print(f'[epoch {epoch+1}/{epochs}], train loss: {train_loss}, train acc: {train_acc}')
        test_acc = evaluate_accuracy_gpu(net, test_iter, device)
        print(f'[test acc: {test_acc}')
        
    

In [11]:
lr, epochs = 0.3, 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu' )
optimizer = optim.SGD(LeNet.parameters(), lr)
loss = nn.CrossEntropyLoss()
train_ch6(LeNet, train_iter, test_iter, epochs, lr, device)

training on cuda


100%|██████████| 235/235 [00:05<00:00, 39.41it/s]


[epoch 1/20], train loss: 0.009061122313141823, train acc: 0.0997


100%|██████████| 40/40 [00:00<00:00, 51.55it/s]


[epoch 1/20], test acc: 0.1


...

[epoch 19/20], test acc: 0.8075


100%|██████████| 235/235 [00:05<00:00, 45.19it/s]


[epoch 20/20], train loss: 0.0019294003723189235, train acc: 0.816


100%|██████████| 40/40 [00:00<00:00, 54.94it/s]

[epoch 20/20], test acc: 0.7774
